## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [ ]:
# python
print("{}+{}={}".format(1, 2, 3)) # {}에 1대1로 래핑되어 출력됨

1+2=3


In [7]:
# prompt 생성
prompt = prompt_template.format(country="대한민국") # 1:1로 래핑된 부분이 country로 대체됨
prompt

'대한민국의 수도는 어디인가요?'

In [8]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [10]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [12]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [ ]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input) # invoke 메서드 호출 : 한번에 모든 결과를 반환.

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배울 수 있도록 많은 예시(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 모으는 거죠.\n\n2. **모델 설계**: 그 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하려고 해요.\n\n3. **학습 과정**:\n   - 모델이 처음에는 무작위로 예측을 해요.\n   - 그리고 그 예측이 맞는지 틀린지 판단하는 '오차'라는 것을 계산해요.\n   - 오차가 크면, 모델이 잘못 배운 부분을 수정하기 위해 '학습'을 합니다.\n   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 예측할 수 있게 돼요.\n\n4. **최적화**:\n   - 이때 '경사 하강법' 같은 방법을 써서, 오차를 줄이는 방향으로 모델의 내부 값을 조정해요.\n   - 이렇게 계속 수정하면서, 모델은 데이터를 더 잘 이해하게 돼요.\n\n5. **완성**:\n   - 충분히 학습이 되면, 새로운 데이터(예를 들어, 새로운 고양이 사진)가 들어와도 올바르게 분류할 수 있게 돼요.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정으로 학습합니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 368, 'prompt_tokens': 22, 'total_tokens': 390, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'au

아래는 스트리밍을 출력하는 예시 입니다.

In [14]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer) # stream_response() 호출 : 결과가 생성될때마다 실시간으로 출력

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 많이 모으는 거죠.

2. **모델 설계**: 그 다음, 이 데이터를 이해하고 분류할 수 있도록 인공지능의 구조(모델)를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 여러 층으로 이루어진 '신경망'이라고 불러요.

3. **학습 과정**:
   - 인공지능은 처음에 무작위로 예측을 해요.
   - 그리고, 정답(예를 들어, 사진이 고양이인지 강아지인지)을 알려주면, 인공지능은 자신의 예측이 얼마나 맞았는지 평가해요.
   - 틀린 정도(오차)를 줄이기 위해, 인공지능은 내부의 '가중치'라는 값을 조금씩 조정해요.
   - 이 과정을 여러 번 반복하면서, 인공지능은 점점 더 정확하게 사진을 구분할 수 있게 되는 거죠.

4. **최적화**: 이때, 오차를 최소화하는 방법(알고리즘)을 사용해서 가중치를 계속 조정해요. 이렇게 해서 인공지능이 더 잘 학습하게 되는 거예요.

5. **완성**: 충분히 학습이 되면, 인공지능은 새로운 사진이 들어왔을 때도 고양이인지 강아지인지 잘 맞출 수 있게 돼요.

요약하자면, 인공지능은 많은 예제와 반복 학습을 통해서, 예측을 더 잘할 수 있도록 내부 값을 조정하는 과정을 거쳐서 배우는 거랍니다!

### 출력파서(Output Parser)

### 10. 출력 파서 (Output Parser)
LLM이 생성한 답변을 우리가 원하는 **특정 형식으로 변환**해주는 도구예요. 예를 들어, LLM이 자유로운 문장으로 답변을 내놓았을 때, 이 답변을 **JSON 형식**이나 **리스트 형식**으로 바꿔주는 역할을 하죠.

In [24]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [30]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

# output_parser를 쓰지 않을 경우 출력되는 모든 메시지를 반환
# output_parser를 쓰면 원하는 답변만을 출력되도록 함.

"물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배울 수 있도록 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 고양이 이름이 적힌 목록이 될 수 있어요.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하려고 해요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 무작위로 예측하거나 판단을 해요.\n   - 그리고 실제 정답(예를 들어, 사진이 고양이인지 아닌지)을 알고, 그 차이(오차)를 계산해요.\n   - 이 오차를 줄이기 위해 모델의 내부 값(파라미터)을 조금씩 조정합니다. 이 과정을 여러 번 반복하면서 모델이 점점 더 정확하게 예측하게 되는 거예요.\n\n4. **최적화**: 이때 '경사 하강법' 같은 방법을 사용해서 오차를 최소화하는 방향으로 파라미터를 조정합니다.\n\n5. **학습 완료**: 여러 번 반복해서 학습하면, 모델은 새로운 데이터에 대해서도 잘 예측할 수 있게 돼요.\n\n쉽게 말하면, 인공지능은 많은 예제들을 보고 '이것이 정답이다'라고 배우면서 점점 더 똑똑해지는 과정입니다. 마치 우리가 새로운 것을 배울 때 여러 번 연습하는 것과 비슷하죠!"

In [22]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.

2. **모델 설계**: 그 다음, 컴퓨터가 데이터를 이해하고 학습할 수 있도록 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하는 방식이에요.

3. **학습 과정**:
   - 모델은 처음에는 무작위로 예측하거나 판단을 해요.
   - 그 예측이 맞는지 틀린지 평가하는 '손실 함수'라는 기준이 있어요.
   - 틀린 정도를 계산해서, '이걸 어떻게 고칠까?' 고민하며 모델의 내부 값(가중치)을 조금씩 조정해요.
   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 예측하게 됩니다.

4. **최적화**:
   - 이때 '경사 하강법' 같은 방법을 사용해서, 손실 값을 최소화하는 방향으로 가중치를 조정해요.
   - 이렇게 해서 모델이 더 좋은 예측을 할 수 있도록 학습하는 거죠.

5. **완성**:
   - 충분히 학습이 되면, 모델은 새로운 데이터(예를 들어, 새로운 고양이 사진)가 들어왔을 때도 잘 맞출 수 있게 돼요.

요약하자면, 인공지능은 많은 데이터를 보고, 예측을 하고, 틀린 점을 고치면서 점점 더 똑똑해지는 과정입니다!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [ ]:
template = """
# 페르소나 설정
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}  # 1번

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [33]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 공원에서 산책하는 애견인과 대화하고 싶어요"}))    
# 위의 셀의 1번{question}의 이름과 같아야함.

- 영어 회화:  
Hello! Your dog looks so happy. How long have you been walking here today?  

- 한글 해석:  
안녕하세요! 당신의 개가 정말 행복해 보여요. 오늘 여기서 얼마나 오래 산책하고 계세요?


In [36]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 돈까스를 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
Waiter: Good evening! Welcome to our restaurant. What would you like to order?
Customer: Hi, I’d like to order the pork cutlet, please.
Waiter: Certainly. Would you like it with rice or bread?
Customer: With rice, please. Thank you.

- 한글 해석:
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 무엇을 주문하시겠어요?
고객: 안녕하세요, 돈까스를 주문하고 싶어요.
웨이터: 알겠어요. 밥과 함께 하시겠어요, 아니면 빵과 함께 하시겠어요?
고객: 밥과 함께 주세요. 감사합니다.

In [30]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
A: Hi! I’d like to order a large pepperoni pizza, please.  
B: Sure! Would you like anything else? Maybe some drinks or a side?  
A: Yes, I’ll also have a medium soda and a side of garlic bread.  
B: Great. Your total is $20.50. Will that be for here or to go?  
A: To go, please.  
B: Okay, your order will be ready in about 15 minutes. Thank you!  

- 한글 해석:  
A: 안녕하세요! 대형 페퍼로니 피자를 주문하고 싶어요.  
B: 네! 다른 것도 필요하세요? 음료수나 사이드도요?  
A: 네, 중간 크기 소다와 마늘빵도 하나 주세요.  
B: 알겠습니다. 총 금액은 $20.50입니다. 포장하시나요, 아니면 여기서 드시겠어요?  
A: 포장해 주세요.  
B: 알겠습니다. 15분 후에 준비될 겁니다. 감사합니다!